In [1]:
import pandas as pd
import numpy as np
import random
import os
from scipy.stats import norm
import matplotlib.pyplot as plt

$B_{call}$：触及强赎条款的`强赎债券价格`；<br>
$B_{put}$：触及(向下)回售条款的`强制回售债券价格`；<br>
$P_{call}$：触及强赎条款的`股票价格`；<br>
$P_{reset}$：触及向下修正条款的`股票价格`；<br>
$P_{call}^{r}$：确定向下调整以后，再度触及强赎条款的`股票价格`，R.V.但是可能可以通过$\theta \in (0,1)$给出大致范围；<br>
$P_{put}$：触及(向下)回售条款的`股票价格`；<br>
$P_{put}^{r}$：确定向下调整以后，再度触及(向下)回售条款的`股票价格`，R.V.是所有这些股票价格中的**最小值**；<br>
$K$：初始设定的转股条款中的`转股股票价格`；<br>
$\delta K=P_{reset}$，$\theta K=K^{r}$；<br>
$F$：债券发行面值，是个固定值100；<br>
$P_{call}^{r} = \theta P_{call}$，$P_{put}^{r} = \theta P_{put}$<br>

$X_{t} = ln(\frac{S_{t}}{P_{reset}})$：$S_{t}$是时点t下的股票随机路径价格，该形式便于套用随机积分的一些结果$\Rightarrow R.V.$；<br>
$X_{t}^{r} = ln(\frac{S_{t}}{P_{put}^{r}})$：$\Rightarrow R.V.$；<br>
$X_{\tau_{call}} = ln(\frac{P_{call}}{P_{reset}})$：是个基于股票价格的比例对数；<br>
$X_{\tau_{reset}} = 0$：根据形式安排可以自然得到0值；<br>
$X_{\tau_{call}^{r}}^{r} = ln(\frac{P_{call}^{r}}{P_{put}^{r}})$：是个基于向下调整后股票价格(R.V.)的比例对数，但是恰有$\Rightarrow C$，一般为$ln(\frac{1.30}{0.70})$；<br>
$X_{\tau_{put}^{r}}^{r} = 0$：根据形式安排可以自然得到0值；<br>
$X_{\tau_{reset}}^{r} = ln(\frac{S_{\tau_{reset}}}{P_{put}^{r}}) = ln(\frac{P_{reset}}{P_{put}^{r}})$：分子是个定值，分母是个关于$\theta$锚定的值，但是恰有$\Rightarrow C$，一般为$ln(\frac{1}{0.70})$；<br>
**`一般赎回触发比例有1.3、1.2，向下修正触发比例有0.9、0.85、0.8，回售触发比例有0.7、0.6.`**

$h_{1} = ln(\frac{K}{P_{reset}})$：`转股股票价格`相对于向下修正条款中的`股票价格`的比例对数，一般为$ln(\frac{1}{0.85})$；<br>
$h_{2} = ln(\frac{P_{call}}{P_{reset}})$：强赎条款中的`股票价格`相对于向下修正条款中的`股票价格`的比例对数，一般为$ln(\frac{1.30}{0.85})$；<br>
$h_{1}^{r} = ln(\frac{\theta K}{P_{put}^{r}}) = ln(\frac{\theta K}{\theta P_{put}}) = ln(\frac{K}{P_{put}})$：两个R.V.的比例对数，但是恰有$\Rightarrow C$，一般为$ln(\frac{1}{0.80})$；<br>
$h_{2}^{r} = ln(\frac{P_{call}^{r}}{P_{put}^{r}}) = ln(\frac{\theta P_{call}}{\theta P_{put}}) = ln(\frac{P_{call}}{P_{put}})$：两个R.V.的比例对数，但是恰有$\Rightarrow C$，一般为$ln(\frac{1.30}{0.70})$；<br>

转移概率：<br>
$p(t_{1},y_{1};t_{2},y_{2})=\frac{2}{l}exp\left(\frac{\mu_{Y}(y_{2}-y_{1})}{\sigma_{Y}^{2}}\right)\sum\limits_{k=1}^{\infty}exp(-\lambda_{Y_{k}}(t_{2}-t_{1}))sin(k\pi\frac{y_{1}}{l})sin(k\pi\frac{y_{2}}{l})$ $\hspace*{1em}$ **(A.2)**
<br>
这里的 $\lambda_{Y_{k}}$ 只出现过一次，并且是作为拉普拉斯变换中的一个中间量起作用的，因此 `k` 实际上是没有固定值的，而是一个代表遍历的(中间)参数。<br>
$$\lambda_{Y_{k}} =\frac{\frac{\mu_{Y}^{2}}{\sigma_{Y}^{2}}+\frac{k^{2}\pi^{2}\sigma_{Y}^{2}}{l^{2}}}{2} $$
其中$y_{1}$是在初始时间的股价位置，$y_{2}$的设定可以根据需要来确定，比如设定为 $l$ 或者0

In [2]:
f = pd.read_csv('C:\\Users\\huangrz\\Desktop\\cb_p\\st_close.csv',
                index_col=[0],
                encoding='GBK',
                low_memory=False)
f.astype(np.float64)
starr = f.iloc[10]
lnarr = np.subtract(np.log(starr),np.log(starr.shift(1)))

In [3]:
global mu, mu_tilde ,sigma
r, sigma = lnarr.mean(), lnarr.std()
mu = r - sigma**2/2
mu_tilde = np.sqrt(mu**2+2*sigma**2*r)

In [4]:
# Valuing resettable convertible bonds: Based on path decomposing
# Article Author: Yun Feng, Bing-hua Huang, Yu Huang
# https://www.sciencedirect.com/science/article/pii/S154461231630157X
global cr, rr, pr
global Xt, Xtr, Xrr, h1, h2, h1r, h2r, pi, e
global F ,Bc ,Bp ,K ,St
cr, rr, pr = 1.3, 0.85, 0.7
F, Bc, Bp = 100, 115, 90
K  = 33.33
St = 34.5
S0 = 30
P_reset = S0*0.85
P_pr = P_reset*0.70 
Xt  = np.log(St/P_reset)
Xtr = np.log(St/P_pr)
Xrr = np.log(1/0.7)
h1  = np.log(1/0.85)
h2  = np.log(1.3/0.85)
h1r = np.log(1/0.7)
h2r = np.log(1.3/0.7)
pi, e = np.pi, np.e

In [5]:
def lambdaYk(mu,sigma,h,k):
    res = (np.divide(mu,sigma)**2+np.divide(k*pi*sigma,h)**2)/2
    return res

# 优化时可以用 sigma2 代替 每次运算 sigma**2
def gu_calc(k, Lambda, t1, t2, X, h):
    prob = np.divide(e**(-Lambda*(t2-t1)),Lambda) * (k*pi) * np.sin(k*pi*(h-X)/h)
    return prob

def gd_calc(k, Lambda, t1, t2, X, h):
    prob = e**(-Lambda*(t2-t1)) * (k*pi) * np.sin(k*pi*X/h)
    return prob

def Z(alpha, mu, sigma, t1, t2, X1, X2, h, n):
    series = 0
    for k in range(n):
        lamyk = lambdaYk(mu,sigma,h,k)
        lval = e**(-lamyk*(t2-t1)) * np.sin(k*pi*X2/h)
        rval = np.divide((mu/sigma**2+alpha)*np.sin(k*pi*X2/h)\
                         -(k*pi/h)*np.cos(k*pi*X2/h),
                         (mu/sigma**2+alpha)**2+(k*pi/h)**2)
        series += lval*rval
    res = (2/h) * e**(mu*(X2-X1)/sigma**2+alpha*X2) * series
    return res

def ex_option_1(t1, t2, n):
    series = 0
    for k in range(n):
        lamyk = lambdaYk(mu_tilde,sigma,h2,Xt)
        series += gu_calc(k,lamyk,t1,t2,Xt,h2)
    lval = e**(mu*(h2-Xt)/sigma**2)
    rval = np.divide(np.sinh(Xt*mu_tilde)/sigma**2,
                     np.sinh(h2*mu_tilde)/sigma**2) - (sigma/h2)**2*series
    res = (max(F*cr,Bc)-F)*lval*rval
    return res

def ex_option_2(r, t1, t2, n):
    discount = e**(-r*(t2-t1))
    res = F*rr*(Z(1, mu, sigma, t1, t2, Xt, h2, h2, n) \
                -Z(1, mu, sigma, t1, t2, Xt, h1, h2, n)) \
        - F*(Z(0, mu, sigma, t1, t2, Xt, h2, h2, n) \
             -Z(0, mu, sigma, t1, t2, Xt, h1, h2, n))
    res = discount * res
    return res

def ex_option_3(t1, t2, n):
    E = 0
    for tau in range(t1,t2+1):
        series1 = 0
        series2 = 0
        for k in range(n):
            lamyk = lambdaYk(mu_tilde,sigma,h2,Xt)
            series1 += gu_calc(k,lamyk,tau,t2,Xrr,h2r)
            series2 += gd_calc(k,lamyk,t1,tau,Xt,h2)
        lval = (sigma/h2)**2 * e**(mu*(h2r-Xrr-Xt)/sigma**2)
        mval = -(sigma/h2r)**2 * series1 \
                + np.divide(np.sinh(Xrr*mu_tilde)/sigma**2,
                            np.sinh(h2r*mu_tilde)/sigma**2)
        rval = series2
        E += lval*mval*rval
    res = (max(F*cr,Bc)-F)*E
    return res

def ex_option_4(r, t1, t2, n):
    discount = e**(-r*(t2-t1))
    integral = 0
    for tau in range(t1,t2+1):
        z1 = Z(1,mu,sigma,tau,t2,Xrr,h2r,h2r,n)
        z2 = Z(1,mu,sigma,tau,t2,Xrr,h1r,h2r,n)
        z3 = Z(0,mu,sigma,tau,t2,Xrr,h2r,h2r,n)
        z4 = Z(0,mu,sigma,tau,t2,Xrr,h1r,h2r,n)
        for k in range(n):
            lamyk = lambdaYk(mu,sigma,h2,k)
            series = gd_calc(k,lamyk,t1,tau,Xt,h2)
        integral += (cr*(z1-z2)-(z3-z4)) * series
    res = discount * F * (sigma/h2)**2 * e**(-mu*Xt/sigma**2) * integral
    return res

# 不知道为啥折现的操作中还要添加一个 rc 的折现利率部分，不予处理(i.e. rc=0).
def ex_option_5(rc, t1, t2, n):
    integral = 0
    for tau in range(t1,t2+1):
        for k in range(n):
            lamyk = lambdaYk(mu_tilde,sigma,h2,k)
            # 由于形式限制，需要对输入变量作一个小调整(变换).
            series1 = gu_calc(k,lamyk+rc,tau,t2,h2r-Xrr,h2r)
            series2 = gd_calc(k,lamyk+rc,t1,tau,Xt,h2)
        lval = -(sigma/h2r)**2 * series1\
                + np.divide(np.sinh(mu_tilde*(h2r-Xrr)/sigma**2),
                            np.sinh(mu_tilde*h2r/sigma**2))
        rval = series2
        integral += lval*rval
    E = (sigma/h2)**2 * e**(-mu*(Xrr+Xt)/sigma**2) * integral
    res = (Bp-F) * E
    return res

In [6]:
# 在v1和v3部分发生了数值爆炸，v5为负数是合理的，因为代表正股长跌
# 触发回售的溢价，其算法的数值基础就是回售价格减去面值(一般为负).
v1=ex_option_1(0,60,1000)
v2=ex_option_2(0.03/365,0,60,1000)
v3=ex_option_3(0,60,1000)
v4=ex_option_4(0.03/365,0,60,100)
v5=ex_option_5(0,0,60,100)
print(v1,v2,v3,v4,v5)

18167.50248708102 -4.725027176143497 872416.7377665306 5.293501702380277 -6.017506921820259
